In [2]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import re
import pyreadr
import random
from itertools import combinations
from collections import OrderedDict, Counter
import matplotlib.pyplot as plt
import torch
from torch import nn, einsum
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from einops import rearrange, repeat, reduce
from einops.layers.torch import Rearrange
from torchmetrics import Accuracy
accuracy = Accuracy()

## Read data

In [2]:
#### Setting
DEVICE = 'cpu' # or 'cuda:0'

exp_name = 'PXD002892'
exp_cond = 'SEC2-heavy'
cur_cv = 'cv1'
feat_type = ['epf_200', 'seq_FCGR_16x']
emb_size = [int(re.sub('x', '', i.split('_')[-1])) for i in feat_type]
emb_size[1] = (emb_size[1])**2

load_dir1 = '/'.join(['path to input', exp_name])
load_dir2 = '/'.join(['path to EPF-data', exp_name])

In [ ]:
#### Read data
train_edge    = torch.tensor( np.load(load_dir1 + '/ref_edge_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
heldout_edge  = torch.tensor( np.load(load_dir1 + '/heldout_edge_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
delfold_edge  = torch.tensor( np.load(load_dir1 + '/delfold_edge_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
exp_edge      = torch.tensor( np.load(load_dir1 + '/exp_edge_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
train_label   = torch.tensor( np.load(load_dir1 + '/ref_label_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
heldout_label = torch.tensor( np.load(load_dir1 + '/heldout_label_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
delfold_label = torch.tensor( np.load(load_dir1 + '/delfold_label_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
train_mask    = torch.tensor( np.load(load_dir1 + '/train_mask_' + exp_cond + '.npz')['arr_0'][int(re.sub('cv', '', 'cv1'))-1] ).to(torch.bool).to(DEVICE)
test_mask     = torch.tensor( np.load(load_dir1 + '/test_mask_' + exp_cond + '.npz')['arr_0'][int(re.sub('cv', '', 'cv1'))-1] ).to(torch.bool).to(DEVICE)

print(train_edge.shape, heldout_edge.shape, delfold_edge.shape, exp_edge.shape)
print(train_label.shape, heldout_label.shape, delfold_label.shape)
print(train_mask.shape, test_mask.shape)

In [5]:
epf = list(pyreadr.read_r(load_dir2 + '/' + exp_cond + '.rds').values())[0].values
pad = np.zeros((epf.shape[0], int(feat_type[0].split('_')[-1]) - epf.shape[1]))
print(epf.shape)
print(pad)

epf = torch.tensor( np.concatenate([epf, pad], 1).reshape(-1, int(feat_type[0].split('_')[-1]), 1)).to(torch.float32).to(DEVICE)
print(epf.shape)

(4002, 55)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
torch.Size([4002, 200, 1])


In [5]:
seq_in_exp_idx     = torch.tensor( np.load(load_dir1 + '/seq_in_exp_idx_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
seq_off_exp_idx    = torch.tensor( np.load(load_dir1 + '/seq_off_exp_idx_' + exp_cond + '.npz')['arr_0'] ).to(torch.long).to(DEVICE)
seq                = torch.tensor( np.load(load_dir1 + '/feature_' + feat_type[1] + '_' + exp_cond + '.npz')['arr_0'] ).to(torch.float32)
seq                = torch.unsqueeze(seq, -1).to(DEVICE)

print(seq_in_exp_idx.shape, seq_off_exp_idx.shape)
print(seq.shape)

torch.Size([3953]) torch.Size([49])
torch.Size([3953, 256, 1])


In [6]:
#### Create val mask
val_idx = random.sample((train_mask==1).nonzero().reshape(-1).tolist(), round(int((train_mask==1).sum())*0.2))
train_idx = list(set((train_mask==1).nonzero().reshape(-1).tolist())^set(val_idx))
print(val_idx[:5], len(val_idx))
print(train_idx[:5], len(train_idx))
print('*'*50)

val_mask = torch.zeros(train_mask.shape[0])
val_mask[val_idx] = 1
val_mask = val_mask.bool()
print(val_mask, val_mask.sum())
print('*'*50)

train_mask = torch.zeros(train_mask.shape[0])
train_mask[train_idx] = 1
train_mask = train_mask.bool()
print(train_mask, train_mask.sum())

[19440, 17750, 21667, 13327, 32241] 5645
[0, 1, 3, 5, 6] 22581
**************************************************
tensor([False, False, False,  ..., False, False, False]) tensor(5645)
**************************************************
tensor([ True,  True, False,  ...,  True, False,  True]) tensor(22581)


## Model layer

In [16]:
class embedding_layer(nn.Module):
    def __init__(self, dim_1, dim_2):
        super(embedding_layer, self).__init__()
        self.Emb = nn.Embedding(dim_1, dim_2, max_norm=True)
    
    def forward(self, in_idx, off_idx, in_emb):
        in_emb = in_emb.squeeze(-1)
        all_tokens = len(list(set(in_idx.tolist() + off_idx.tolist())))
        out_emb = self.Emb(torch.arange(all_tokens))
        
        past_off_idx = []
        for idx in range(all_tokens):
            if idx in in_idx:
                out_emb[idx, :] += in_emb[(idx - len(past_off_idx)), ]
            else:
                past_off_idx.append(idx)
        return out_emb.unsqueeze(-1)

In [17]:
class Encoder(nn.Module):
    def __init__(self, hid_dim=8, dropout=0.1, head_num=8, head_dim=32, max_size=emb_size*2):
        super().__init__()
        
        self.hid_dim = hid_dim
        
        self.conv = nn.Conv1d(1, hid_dim, 3, padding=1)
        self.fc1 = nn.Linear(max_size*(hid_dim + 1), 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        
        self.dropout = dropout
        
    def forward(self, inp, e_idx, bias=None):
        x = self.conv(inp.permute(0, 2, 1)).permute(0, 2, 1) + inp.repeat(1, 1, self.hid_dim)
        
        x_e1 = inp[e_idx[:, 0]] - inp[e_idx[:, 1]]
        x_e2 = x[e_idx[:, 0]] - x[e_idx[:, 1]]
        x_e = torch.cat((x_e1 , x_e2), -1).flatten(start_dim=1)
        
        x_e = F.dropout(F.relu(self.fc1(x_e)), p=self.dropout)
        x_e = F.dropout(F.relu(self.fc2(x_e)), p=self.dropout)
        w = torch.sigmoid(self.fc3(x_e)).squeeze()
        feat = x.flatten(start_dim=1)
        
        if bias != None:
            feat += bias
        
        return feat, w

In [18]:
class Encoderr(nn.Module):
    def __init__(self, hid_dim=16, max_size=emb_size, dropout=0.1, all_tokens=20):
        super().__init__()
        self.dropout = dropout
        
        self.emb = embedding_layer(all_tokens, emb_size[1])
        self.enc = Encoder(hid_dim=hid_dim, max_size=sum(max_size), dropout=dropout)
        
    def forward(self, inp_e, inp_s, e_idx, in_list=None, off_list=None):
        
        if off_list is not None:
            inp_s = self.emb(in_list, off_list, inp_s)
        
        inp = torch.cat([inp_e, inp_s], 1)
        x, weight = self.enc(inp, e_idx, bias=None)
        
        return x, weight, inp[:, inp_e.shape[1]:, :]

## Train

In [19]:
predict_weight    = 2.5
L2_weight         = 0.25
weight_decay      = 2.0e-3

model = Encoderr(hid_dim=16, max_size=emb_size, dropout=0.3, all_tokens=max(seq_in_exp_idx.tolist() + seq_off_exp_idx.tolist()) + 1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=weight_decay)

In [20]:
out_path = './performance_record'
if not os.path.exists(out_path):
      os.makedirs(out_path)

out_path = './best_model_weight'
if not os.path.exists(out_path):
      os.makedirs(out_path)

out_path = './output'
if not os.path.exists(out_path):
      os.makedirs(out_path)

In [ ]:
train_loss_list = open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'train_loss_list']),'w')
val_loss_list = open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'val_loss_list']),'w')
train_acc_list = open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'train_acc_list']),'w')
val_acc_list = open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'val_acc_list']),'w')
best_val_loss = float('inf')

for epoch in range(100):
    
    #### Train =============================================================================================================================
    model.train()
    optimizer.zero_grad()
    
    _, train_label_predict, _ = model(epf, seq, train_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
    
    train_loss = F.binary_cross_entropy(train_label_predict[train_mask], train_label[train_mask].to(torch.float32))
    L2_loss = sum([ (v**2).sum()/2 for v in model.parameters() ])
    
    loss = predict_weight*train_loss + weight_decay*L2_weight*L2_loss
    
    loss.backward()
    optimizer.step()
    
    train_acc = accuracy(train_label_predict[train_mask] > 0.5, train_label[train_mask] > 0.5)
    
    #### Validataion =============================================================================================================================
    model.eval()
    _, train_label_predict, _ = model(epf, seq, train_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
    
    val_loss = F.binary_cross_entropy(train_label_predict[val_mask], train_label[val_mask].to(torch.float32))
    val_acc = accuracy(train_label_predict[val_mask] > 0.5, train_label[val_mask] > 0.5)
    
    train_loss_list.write(str(train_loss.item()))
    val_loss_list.write(str(val_loss.item()))
    train_acc_list.write(str(train_acc.item()))
    val_acc_list.write(str(val_acc.item()))
    
    train_loss_list.write('\n')
    val_loss_list.write('\n')
    train_acc_list.write('\n')
    val_acc_list.write('\n')
    
    print("Epoch:", '%04d' % (epoch + 1), "all_loss=", "{:.4f}".format(loss))
    print("train_loss=", "{:.4f}".format(predict_weight*train_loss), "val_loss=", "{:.4f}".format(predict_weight*val_loss),
          "lossL2=", "{:.4f}".format(weight_decay*L2_weight*L2_loss),
          "train_acc=", "{:.4f}".format(train_acc), "val_acc=", "{:.4f}".format(val_acc))
    
    if val_loss < best_val_loss:
        torch.save(model.state_dict(), './best_model_weight/' + '_'.join([exp_cond, cur_cv, 'stat_dict']))
        best_val_loss = val_loss
        
train_loss_list.close()
val_loss_list.close()
train_acc_list.close()
val_acc_list.close()

## Load best model

In [13]:
model_dict = torch.load('./best_model_weight/' + '_'.join([exp_cond, cur_cv, 'stat_dict']))
model_load = Encoderr(hid_dim=16, max_size=emb_size, dropout=0.3, all_tokens=max(seq_in_exp_idx.tolist() + seq_off_exp_idx.tolist()) + 1).to(DEVICE)
model_load.load_state_dict(model_dict)
model_load.eval()

Encoderr(
  (emb): embedding_layer(
    (Emb): Embedding(2019, 256, max_norm=True)
  )
  (enc): Encoder(
    (conv): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (fc1): Linear(in_features=7752, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
#### train edge
_, train_pred, _ = model_load(epf, seq, train_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
print((train_pred > 0.5).sum())
print((train_pred > 0.5).sum() / train_pred.shape[0])
print('*'*25)

print( (((train_pred > 0.5).to(torch.float32) + (train_label > 0.5).to(torch.float32))==2).sum() / (train_label > 0.5).sum() )
print( (((train_pred <= 0.5).to(torch.float32) + (train_label <= 0.5).to(torch.float32))==2).sum() / (train_label <= 0.5).sum() )
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'train', 'out']), train_pred.detach().numpy())

In [ ]:
train_loss = F.binary_cross_entropy(train_pred[train_mask], train_label[train_mask].to(torch.float32))
val_loss = F.binary_cross_entropy(train_pred[val_mask], train_label[val_mask].to(torch.float32))
test_loss = F.binary_cross_entropy(train_pred[test_mask], train_label[test_mask].to(torch.float32))
train_acc = accuracy(train_pred[train_mask] > 0.5, train_label[train_mask] > 0.5)
val_acc = accuracy(train_pred[val_mask] > 0.5, train_label[val_mask] > 0.5)
test_acc = accuracy(train_pred[test_mask] > 0.5, train_label[test_mask] > 0.5)
print(train_loss, val_loss, test_loss)
print(train_acc, val_acc, test_acc)

In [ ]:
#### heldout edge
_, test_pred, _ = model_load(epf, seq, heldout_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
print((test_pred > 0.5).sum())
print((test_pred > 0.5).sum() / test_pred.shape[0])
print('*'*25)

print( (((test_pred > 0.5).to(torch.float32) + (heldout_label > 0.5).to(torch.float32))==2).sum() / (heldout_label > 0.5).sum() )
print( (((test_pred <= 0.5).to(torch.float32) + (heldout_label <= 0.5).to(torch.float32))==2).sum() / (heldout_label <= 0.5).sum() )
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'heldout', 'out']), test_pred.detach().numpy())

In [ ]:
#### delfold edge
_, del_pred, _ = model_load(epf, seq, delfold_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
print((del_pred > 0.5).sum())
print((del_pred > 0.5).sum() / del_pred.shape[0])
print('*'*25)

print( (del_pred > 0.5).sum() / delfold_label.shape[0] )
print( (((del_pred <= 0.5).to(torch.float32) + (delfold_label <= 0.5).to(torch.float32))==2).sum() / delfold_label.shape[0] )
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'delfold', 'out']), del_pred.detach().numpy())

In [ ]:
#### exp edge
_, exp_pred, _ = model_load(epf, seq, exp_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
print((exp_pred > 0.5).sum())
print((exp_pred > 0.5).sum() / exp_pred.shape[0])
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'exp', 'out']), exp_pred.detach().numpy())

In [ ]:
#### Emb of all edge
all_edge = torch.cat([train_edge, heldout_edge, delfold_edge, exp_edge], 0)
print(all_edge.shape)

emb, all_pred, emb_seq = model_load(epf, seq, all_edge, in_list=seq_in_exp_idx, off_list=seq_off_exp_idx)
print((all_pred > 0.5).sum())
print((all_pred > 0.5).sum() / all_pred.shape[0])
print(emb.shape, emb_seq.shape)

np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'all', 'out']), all_pred.detach().numpy())
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'all_emb', 'out']), emb.detach().numpy())
np.savez('./output/' + '_'.join([exp_cond, cur_cv, 'seq_emb', 'out']), emb_seq.detach().numpy())

## Plot loss

In [27]:
with open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'train_loss_list'])) as f:
    train_loss_list = [float(line.rstrip('\n')) for line in f]
with open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'val_loss_list'])) as f:
    val_loss_list = [float(line.rstrip('\n')) for line in f]
with open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'train_acc_list'])) as f:
    train_acc_list = [float(line.rstrip('\n')) for line in f]
with open('./performance_record/' + '_'.join([exp_cond, cur_cv, 'val_acc_list'])) as f:
    val_acc_list = [float(line.rstrip('\n')) for line in f]

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15, 5))
ax[0].plot(train_acc_list, 'b', label = 'train_acc')
ax[0].plot(val_acc_list, 'y', label = 'val_acc')
ax[0].legend()
ax[1].plot(train_loss_list, 'b', label = 'train_loss')
ax[1].plot(val_loss_list, 'y', label = 'val_loss')
ax[1].legend()
fig.suptitle(' '.join([org, exp_name, exp_cond, cur_cv]), fontsize=16)
plt.show()